In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import math
#importing the dependencies

In [2]:
def region_of_interest(img,vertices):
    mask=np.zeros_like(img)
    match_mask_color=255
    cv2.fillPoly(mask,vertices,match_mask_color)
    masked_image=cv2.bitwise_and(img,mask)
    return masked_image
#functon for extracting the reigon of interest area in our case a triangular region

In [3]:
def draw_lines(img,lines,color=[255,0,0],thickness=3):
    if lines is None :
        return
    copy=np.copy(img)
    line_image=np.zeros(shape=(copy.shape[0],copy.shape[1],3),dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),color,thickness)
    copy=cv2.addWeighted(img,0.8,line_image,1.0,0.0)
    return copy
#function for drawing the lines 

In [16]:

def pipeline(image):
    """
    An image processing pipeline which will output
    an image with the lane lines annotated.
    """
    height = image.shape[0]
    width = image.shape[1]
    #extracting the width and height for ROI calculation
    
    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    #array of ROI points
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cannyed_image = cv2.Canny(gray_image, 100, 200)
    #extracting the edges from the image
    
    cropped_image = region_of_interest(
        cannyed_image,
        np.array(
            [region_of_interest_vertices],
            np.int32
        ),
    )
    #cropping the image to ROI
    
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )
    #extracting the lines end point from hough space  transformations
    
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    # arrays for seperating left and right lanes
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if math.fabs(slope) < 0.5:
                continue
            if slope <= 0:
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else:
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])
    #main loop for seperating left and right lanes
    
    min_y = int(image.shape[0] * (3 / 5))
    max_y = int(image.shape[0])
    #these are values for plotting the lines y axis points
    
    poly_left = np.poly1d(np.polyfit(
        left_line_y,
        left_line_x,
        deg=1
    ))
 
    left_x_start = int(poly_left(max_y))
    left_x_end = int(poly_left(min_y))
    #processing for the left lines x-axis points
    
    poly_right = np.poly1d(np.polyfit(
        right_line_y,
        right_line_x,
       deg=1
    ))
    right_x_start = int(poly_right(max_y))
    right_x_end = int(poly_right(min_y))
     #processing for the right lines x-axis points
 
    line_image = draw_lines(
        image,
        [[
            [left_x_start, max_y, left_x_end, int(min_y)],
            [right_x_start, max_y, right_x_end, int(min_y)],
        ]],
        thickness=5,
    )
    return line_image
#main function for processing all the frames of the video

In [21]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'solidYellowLeft_output.mp4'
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video solidYellowLeft_output.mp4
[MoviePy] Writing video solidYellowLeft_output.mp4


100%|█████████▉| 681/682 [00:24<00:00, 27.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidYellowLeft_output.mp4 

